MP2

# Imports

In [1]:
import pandas as pd
import torch
import numpy as np
import random

# Loading and splitting the data

We use a 80/20 split for the training and testing data which can be seen by the choices of the slices.

In [19]:
  ratings = pd.read_csv("http://www.cs.toronto.edu/~guerzhoy/324/movielens/ratings.csv")
  ratings_train = ratings.sort_values(by=['timestamp'])[0:80668]
  ratings_test = ratings.sort_values(by=['timestamp'])[80668:-1]

In [20]:
ratings_train

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615
...,...,...,...,...
79752,495,132796,1.0,1458634739
79756,495,139385,5.0,1458634761
79747,495,122882,4.5,1458634764
79566,495,2959,5.0,1458634866


# Part 1: Training with **Embeddings**

We choose to go for one embedding object here for users and movies instead of two.

In [21]:
n_users = len(ratings_train.userId.unique())
n_movies = len(ratings_train.movieId.unique())
n = n_users + n_movies
d = 10
device = 'cuda'
embedding = torch.nn.Embedding(n, d).to(device)

AssertionError: Torch not compiled with CUDA enabled

## Mapping the Movies and Users to Indicies forward and backward

Here we want to access the movies and users as indicies, but then also access the incidices to get the movies and users. This will become very useful for training the data, and also very useful for getting the correct userIDs and movieIDs at the end.

In [5]:
map_movies = {}
map_users = {}
map_idx_user = {}
map_idx_movie = {}

i = 0
for index in ratings_train.index:
  if ratings_train["movieId"][index] not in map_movies:
    map_movies[ratings_train["movieId"][index]] = i
    map_idx_movie[i] = ratings_train["movieId"][index]
    i+=1

i = 0
for index in ratings_train.index:
  if ratings_train["userId"][index] not in map_users:
    map_users[ratings_train["userId"][index]] = i
    map_idx_user[i] = ratings_train["userId"][index]
    i+=1

## Creating the Adjacency Matrix

The adjacency matrix represents where the user rated a movie 5 star as 1, has seen the movie but not rated it 5 star as -1, and not seen as 0

In [6]:
A = torch.zeros((n_users, n_movies))
for index in ratings_train.index:
  if ratings_train["rating"][index] == 5.0:
    A[map_users[ratings_train["userId"][index]]][map_movies[ratings_train["movieId"][index]]] = 1
  else:
    A[map_users[ratings_train["userId"][index]]][map_movies[ratings_train["movieId"][index]]] = -1

In [7]:
A

tensor([[ 1.,  1.,  1.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0., -1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ..., -1., -1.,  1.]])

## Creating the Embeddings for the Movies and Users as an update function

Every time the training loop runs, the new embeddings of the users and movies need to be updated. This function will do that in the training loop.

In [8]:
def update(n_users, n_movies, embedding):
  emb_movies = embedding(torch.arange(n_movies))

  emb_users = embedding(torch.arange(n_movies, n))

  return emb_users, emb_movies

## The Cost function

The cost function is as described in the lab handout. The difference here is that it returns 1/total, since we need to optimize the function given in the handout, we try to reduce 1/total as much as possible.

In [9]:
def cost(n_users, n_movies, emb_users, emb_movies, A):
    total = 0 
    for i in range(len(A)):
      movies_5star = np.where(A[i] == 1)[0]
      movies_4star = np.where(A[i] != 1)[0]

      five_star_sum = torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[i],torch.transpose(emb_movies[movies_5star], 0, 1))))) * 200
      four_star_sum = torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[i],torch.transpose(emb_movies[movies_4star], 0, 1)))))

      total+= (five_star_sum - four_star_sum)

    
    return 1/total

## Training the Embeddings

The training loop takes 1000 epochs, a learning rate of 0.0001, and an Adam optimizer.

In [17]:
epochs = 1000
lr = 0.0001

params = [*embedding.parameters()] 
optimizer = torch.optim.Adam(params, lr = lr)

for epoch in range(epochs):
    emb_users, emb_movies = update(n_users, n_movies, embedding)
    loss = cost(n_users, n_movies, emb_users, emb_movies, A)

    optimizer.zero_grad()           # cleans the gradients
    loss.backward(retain_graph=True)
    optimizer.step()
  
    if epoch % 50 == 0:
      print("epoch = ", epoch, "loss=", loss)

RuntimeError: invalid argument 10: ldb should be at least max(1, 0), but have 0 at C:\Users\builder\AppData\Local\Temp\pip-req-build-6nmox5re\aten\src\TH/generic/THBlas.cpp:368

## Getting 150 highest dot products between movies and users

Here we find the top150 movies per user. I do think this code is the hardest to follow along, so it is commented to explain each line, instead of an overview.

Important to note the "torch.where(A[users] == 0)" which states that the dot products are occuring between users and movies that have NOT been seen already.

In [ ]:
all_comb = torch.matmul(emb_users, torch.transpose(emb_movies, 0 , 1))
# Multiplies the user and movies embeddings matricies together

all_recall = []

for users in range(n_users):
  ind = torch.where(A[users] == 0)[0] 
  # Indicies for the movies that the user has not seen

  user_mul_movie = all_comb[users, ind]
  # Gets the dot product values of all the movies that have not been seen for the user 

  indicies_sorted = torch.argsort(user_mul_movie,-1,True)
  # Sorts these dot products

  if len(indicies_sorted) < 150:
    take_top = len(indicies_sorted)
  else:
    take_top = 150
  # These statements check if there are 150 reccomendations for the movies or less

  sorted_movie_ids = ind[indicies_sorted]
  top_150 = sorted_movie_ids[0:take_top]

  all_recall.append(top_150)
  # Puts the top movie reccomendations in a list where the indicies represent the user

In [ ]:
all_recall

## Dropping Test set rows

All rows that are dropped are the ones where the user or movie do not occur in the training data. These cannot possibly be in the Recall150.

In [ ]:
for index in ratings_test.index:
  if (ratings_test["userId"][index] not in map_users) or (ratings_test["movieId"][index] not in map_movies):
    ratings_test = ratings_test.drop(index = index, axis = 0)

In [ ]:
ratings_test

## Creating a Test set adjacency matrix

This creates the adjacency matrix for the test data. Same procedure as the training adjacency matrix.

In [ ]:
A_test = torch.zeros((n_users, n_movies))
for (i, index) in enumerate(ratings_test.index):
  if ratings_test["rating"][index] == 5:
    A_test[map_users[ratings_test["userId"][index]]][map_movies[ratings_test["movieId"][index]]] = 1
  else:
    A_test[map_users[ratings_test["userId"][index]]][map_movies[ratings_test["movieId"][index]]] = -1

In [ ]:
A_test

## Implimenting @Recall150 (Test Set)

The output is a dictionary containing the user as the key and a tuple (R_u intersection with P_u, @Recall magnitude)


In [ ]:
Recall_150 = {}
for user in range(n_users):
  indicies_movies = torch.where(A_test[user] == 1)[0]
  real_Ru = set()
  real_Pu = set()
  if len(indicies_movies) == 0:
    Recall_150[user+1] = (None, None)
  else:

    for elem in indicies_movies:
      real_Pu.add(map_idx_movie[elem.item()])
    
    for elem in all_recall[user]:
      real_Ru.add(map_idx_movie[elem.item()])

    Recall_150[map_idx_user[user]] = (real_Pu & real_Ru, len(real_Pu & real_Ru)/len(real_Pu))


#Part 2: Node2Vec

A really important note for this part. All the data manipulation and adjacency matrix making has been done. So none of that is included here. The same variables and data is used from the previous part.

New Embedding object

Need to do this to reset the embedding object from the previous part.

In [ ]:
embedding = torch.nn.Embedding(n, d).to(device)

## Random walk function

This part is very hard to follow, I will explain it in the code as to what is going on.

In [ ]:
def node2vec(starting_node,length, p, q, A):

  path = [starting_node]

  user = True
  # Always start the walk on a user node

  BFS_DFS = "DFS"
  # Default to a DFS walk (this actually doesn't do anything, just need to initalize the variable)

  while len(path) < length:
    # Run while the path is less than the given length 

    current_node = path[-1]
    # Take the last node in the path (the node being explored)

    if user == True:
      neighbours = np.where(A[current_node] == 1)
    else:
      neighbours = np.where(A[:,current_node] == 1)

    # Depending on if the node is a user or movie we need to set its neighbours
    # Users will never be neighbours to other users
    # Movies will never be neighbouts to other movies

    breaker = False
    # Initalizing a breaker variable (to get out of the while loop)

    while True:
      if random.random() < 1/p:
        BFS_DFS = "BFS"
        break 
      for i in range(len(neighbours)):
        if random.random() < 1/q:
          BFS_DFS = "DFS"
          breaker = True
          break
      if breaker:
        break

      # This while loop changes the value of BFS_DFS
      # It sets which type of exploration needs to be done 

    if (len(path)) == 1:
      if (len(neighbours[0])) == 0:
        return None
      else:
        path.append(random.choice(neighbours[0]))

    elif (len(path)) > 1:
      if BFS_DFS == "DFS":
        if len(neighbours[0]) == 0:
          path.append(path[-2])
        else:
          path.append(random.choice(neighbours[0]))      
      elif BFS_DFS == "BFS":
        path.append(path[-2])

    # This simply adds the correct node to the path
    # If DFS add a random neighbour, if BFS go back to the original node
    # If the node has no neighbours go ba
    
    user = not(user)
    # Change the variable from user to movie or vise versa
  return path

## Cost of Random walk

This is the cost function described in the handout.

In [ ]:
def cost_node2vec(n_users, n_movies, emb_users, emb_movies, p, q, walk_length):
  walks = []
  total = 0
  for user in range(n_users):
    walks.append(node2vec(user,walk_length, p, q, A))
    if walks[-1] == None:
      continue
    neighbours_user = walks[user][::2]
    neighbours_movie = walks[user][1::2]

    neigh_user_sum = torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[user], torch.transpose(emb_users[neighbours_user],0,1)))))
    neigh_movie_sum = torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[user], torch.transpose(emb_movies[neighbours_movie],0,1)))))

    denominator = torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[user], torch.transpose(emb_users,0,1))))) + torch.sum(torch.log(torch.sigmoid(torch.matmul(emb_users[user], torch.transpose(emb_movies,0,1)))))

    total += neigh_user_sum + neigh_movie_sum - denominator

  return 1/total


## Update the Embeddings

Same update function as before

In [ ]:
def update_node2vec(n_users, n_movies, embedding):  
  emb_movies = embedding(torch.arange(n_movies).to(device))
  emb_users = embedding(torch.arange(n_movies, n).to(device))

  return emb_movies, emb_users

## Train the Node2Vec Embeddings

The training model is similar to the previous one. The main difference here is cost function and the 3 new parameters being p,q and walk_length. These are hyper-parameters and dictate the random-walk from the previous section.

In [ ]:
epochs = 1000
embedding = torch.nn.Embedding(n_users+n_movies, d).to(device)
optimizer = torch.optim.Adam([*embedding.parameters()], lr = 0.0001)

p = 7
q = 10
walk_length  = 7

for epoch in range(epochs):
    emb_movies, emb_users = update_node2vec(n_users, n_movies, embedding)
    cost = cost_node2vec(n_users, n_movies, emb_users, emb_movies, p, q, walk_length)

    optimizer.zero_grad()           # cleans the gradients
    cost.backward(retain_graph=True)
    optimizer.step()
  
    if epoch % 50 == 0:
      print("epoch = ", epoch, "cost=", cost)

## Implimenting @Recall150

Same implimentation as the embeddings

In [ ]:
Recall_150_n2v = {}
for user in range(n_users):
  indicies_movies = torch.where(A_test[user] == 1)[0]
  real_Ru = set()
  real_Pu = set()
  if len(indicies_movies) == 0:
    Recall_150_n2v[user+1] = (None, None)
  else:

    for elem in indicies_movies:
      real_Pu.add(map_idx_movie[elem.item()])
    
    for elem in all_recall[user]:
      real_Ru.add(map_idx_movie[elem.item()])


    Recall_150_n2v[map_idx_user[user]] = (real_Pu & real_Ru, len(real_Pu & real_Ru)/len(real_Pu))

In [ ]:
Recall_150_n2v

# Discussion

So the lab did not go entirely as planned. The results from Part 1 and Part 2 of the lab did not fair well. Most of the code I thought would work properly, hwoever there are most likely some tweaks that causes the Recall150 to not produce the correct results. I do believe with more guidance as to how to impliment certain parts of the code would be beneficial for sure. I did do the best I can with the given time and knowledge of this course.